## Итераторы и генераторы

Реализуем класс, который назовем Repeater, итерации по которому можно выполнять в цикле for-in следующим образом: 

In [1]:
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return self
    def __next__(self):
        return self.value

In [ ]:
repeater = Repeater('Привет')
# это бесконечный цикл:
for item in repeater:
    print(item)

Аналогично:

In [ ]:
repeater = Repeater('Привет')
iterator = repeater.__iter__()
# это бесконечный цикл:
while True:
    item = iterator.__next__()
    print(item)

In [2]:
repeater = Repeater('Привет')
iterator = iter(repeater)
next(iterator)

'Привет'

In [3]:
next(iterator)

'Привет'

Сейчас мы напишем еще один класс итератора, который назовем BoundedRepeater . Он будет похож на предыдущий пример с  Repeater , но на этот раз мы хотим, чтобы он останавливался после предопределенного количества повторений.

Как итератор сигнализирует о том, что он пуст и исчерпал элементы, выдаваемые во время выполнения итераций? 

Создадим простой список с несколькими элементами, а затем будем выполнять его итеративный обход до тех пор, пока он не исчерпает элементы, чтобы увидеть, что произойдет:

In [4]:
my_list = [1, 2, 3]
iterator = iter(my_list)
next(iterator)

1

In [5]:
next(iterator)

2

In [6]:
next(iterator)

3

In [7]:
next(iterator)

StopIteration: 

Если продолжить запрашивать значения из итератора, он продолжит вызывать исключения StopIteration, сигнализируя о том, что больше нет значений, доступных для итераций.

Итераторы Python не могут быть «обнулены», когда они исчерпаны. В этом случае им полагается вызывать исключение StopIteration при каждом вызове функции next(). Чтобы возобновить итерации, нужно запросить свежий объект-итератор при помощи функции iter(). 

In [8]:
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value

Итерации прекращаются после ряда повторений, определенных в параметре max_repeats :

In [9]:
repeater = BoundedRepeater('Привет', 3)
for item in repeater:
    print(item)

Привет
Привет
Привет


Если переписать этот последний пример цикла for...in , устранив часть синтаксического сахара, то в итоге мы получим следующий ниже расширенный фрагмент кода:

In [10]:
repeater = BoundedRepeater('Привет', 3)
iterator = iter(repeater)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        break
    print(item)

Привет
Привет
Привет


При каждом вызове функции next() в этом цикле мы выполняем проверку на исключение StopIteration и при необходимости выходим из цикла while.

# Функции-генераторы

Можно переписать класс Repeater в качестве генератора, он будет выглядеть так: 

In [11]:
def repeater(value):
    while True:
        yield value

Мы только что перешли от семи строк кода к трем. Неплохо, правда? Как видите, генераторы похожи на обычные функции, но вместо инструкции возврата return в них для передачи данных назад источнику вызова используется инструкция yield .

In [ ]:
# бесконечный цикл
for x in repeater('Привет'):
    print(x)

Итак, каким же образом эти генераторы работают? Они похожи на нормальные функции, но их поведение очень различается. Начнем с того, что вызов функции-генератора вообще не выполняет функцию. Он просто создает и возвращает объект-генератор: 

In [12]:
repeater('Эй')

<generator object repeater at 0x7ff5bc283de0>

Программный код в функции-генераторе исполняется только тогда, когда функция next() вызывается с объектом-генератором в качестве аргумента:

In [13]:
generator_obj = repeater('Эй')
next(generator_obj)

'Эй'

Если вы еще раз прочитаете код функции repeater, то увидите, что, судя по всему, ключевое слово yield каким-то образом останавливает эту функцию-генератор посередине исполнения, а затем возобновляет ее на более позднем этапе:

In [14]:
def repeater(value):
    while True:
        yield value

И это вполне подходящая модель того, что здесь происходит. Дело в том, что, когда инструкция return вызывается внутри функции, она безвозвратно передает управление назад источнику вызова функции. Когда же вызывается инструкция yield, она тоже передает управление назад источнику вызова функции — но она это делает лишь временно.

В отличие от инструкции return, которая избавляется от локального состояния функции, инструкция yield приостанавливает функцию и сохраняет ее локальное состояние. На практике это означает, что локальные переменные и состояние исполнения функции-генератора лишь откладываются в сторону и не выбрасываются полностью. Исполнение может быть возобновлено в любое время вызовом функции next() с генератором в качестве аргумента:

In [15]:
iterator = repeater('Привет')
next(iterator)

'Привет'

In [16]:
next(iterator)

'Привет'

Напомним, что в нашем итераторе на основе класса мы смогли подать сигнал об окончании итераций путем вызова исключения StopIteration вручную. Поскольку генераторы полностью совместимы с итераторами на основе класса, за сценой будет по-прежнему происходить то же самое. К счастью, на этот раз мы будем работать с более приятным интерфейсом. Генераторы прекращают порождать значения, как только поток управления возвращается из функции-генератора каким-либо иным способом, кроме инструкции yield. Это означает, что вам больше вообще не нужно заботиться о вызове исключения StopIteration!

Приведу пример:

In [17]:
def repeat_three_times(value):
    yield value
    yield value
    yield value

Обратите внимание: эта функция-генератор не содержит никакого цикла. В действительности она проста как божий день и состоит всего из трех инструкций yield. Если yield временно приостанавливает выполнение функции и передает значение назад источнику вызова, то что произойдет, когда мы достигнем конца этого генератора? Давайте узнаем:

In [18]:
for x in repeat_three_times('Всем привет'):
    print(x)

Всем привет
Всем привет
Всем привет


Как вы, возможно, и ожидали, этот генератор прекратил порождать новые значения после трех итераций. 

Можно предположить, что он это сделал путем вызова исключения StopIteration, когда исполнение достигло конца функции. Но чтобы быть до конца уверенными, давайте подтвердим
это еще одним экспериментом:

In [19]:
iterator = repeat_three_times('Всем привет')
next(iterator)

'Всем привет'

In [20]:
next(iterator)

'Всем привет'

In [21]:
next(iterator)

'Всем привет'

In [22]:
next(iterator)

StopIteration: 

Этот итератор вел себя именно так, как мы и ожидали. Как только мы достигаем конца функции-генератора, он начинает вызывать StopIteration, сигнализируя о том, что у него больше нет значений, которые он мог бы предоставить.

Класс BoundedIterator реализовал итератор, который будет повторять значение, заданное определенное количество раз.

Почему бы не попробовать реализовать класс BoundedRepeater заново как функцию-генератор? Сделаю первую попытку:

In [24]:
def bounded_repeater(value, max_repeats):
    count = 0
    while True:
        if count >= max_repeats:
            return
        count += 1
        yield value

Я преднамеренно сделал цикл while в этой функции несколько громоздким. Я хотел продемонстрировать, как вызов инструкции return из генератора приводит к остановке итераций с исключением StopIteration. Мы вскоре подчистим и еще немного упростим эту функцию-генератор,
но сначала давайте испытаем то, что у нас есть сейчас:

In [25]:
for x in bounded_repeater('Привет', 4):
    print(x)

Привет
Привет
Привет
Привет


Великолепно! Теперь у нас есть генератор, который прекращает порождать значения после настраиваемого количества повторений. Он использует инструкцию yield, чтобы передавать значения назад до тех пор, пока он наконец не натолкнется на инструкцию return и итерации не прекратятся.

Мы можем упростить этот генератор еще больше. Воспользуемся тем, что в конец каждой функции Python добавляет неявную инструкцию return None. И вот как будет выглядеть наша окончательная реализация:

In [26]:
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

Генераторы помогают «абстрагироваться от» большей части шаблонного кода, который в других обстоятельствах был бы необходим во время написания итераторов на основе класса. 

# Выражения-генераторы (generator expressions) добавляют сверху еще один слой синтаксического сахара

Выражения-генераторы представляют собой еще более эффективную краткую форму для создания итераторов. Благодаря простому и сжатому синтаксису, который похож на конструкцию включения в список, вы сможете определять итераторы в одной строке кода.

Приведу пример:

In [27]:
iterator = ('Привет' for i in range(3))

Во время выполнения итераций данное выражение-генератор порождает ту же самую последовательность значений, что и функция-генератор bounded_repeater, которую мы написали в предыдущем разделе.

Разве не удивительно, что однострочное выражение-генератор теперь делает работу, для выполнения которой ранее требовалась четырехстрочная функция-генератор или намного более длинный итератор на основе класса? 

In [30]:
iterator = ('Привет' for i in range(3))
for x in iterator:
    print(x)

Привет
Привет
Привет


Из нашего однострочного выражения-генератора мы получили те же самые результаты, которые мы получали из функции-генератора bounded_repeater.

Правда, есть одно маленькое предостережение: после того как выражение-генератор было использовано, оно не может быть перезапущено или использовано снова. Поэтому в некоторых случаях предпочтительнее использовать функции-генераторы или итераторы на основе класса.

Как вы уже поняли, выражения-генераторы несколько напоминают включения в список:

In [31]:
listcomp = ['Привет' for i in range(3)]
genexpr = ('Привет' for i in range(3))

Однако в отличие от включений в список выражения-генераторы не конструируют объекты-списки. Вместо этого они генерируют значения «точно в срок» подобно тому, как это сделал бы итератор на основе класса или функция-генератор.

Присваивая выражение-генератор переменной, вы просто получите итерируемый «объект-генератор»:

In [32]:
listcomp

['Привет', 'Привет', 'Привет']

In [33]:
genexpr

<generator object <genexpr> at 0x7ff5bc2072a0>

Для того чтобы получить доступ к значениям, порожденным выражением-генератором, вам нужно вызвать с ним метод next() точно так же, как вы бы сделали с любым другим итератором:

In [34]:
next(genexpr)

'Привет'

In [35]:
next(genexpr)

'Привет'

In [36]:
next(genexpr)

'Привет'

In [37]:
next(genexpr)

StopIteration: 

Как вариант, вы также можете вызвать функцию list() c выражением-генератором, в результате чего вы сконструируете объект-список, содержащий все произведенные значения:

In [38]:
genexpr = ('Привет' for i in range(3))
list(genexpr)

['Привет', 'Привет', 'Привет']

Разумеется, это был всего лишь игрушечный пример, который показывает, как можно «преобразовывать» выражение-генератор (или любой другой итератор, если уж на то пошло) в список. Если же вам нужен объект-список прямо на месте, то в большинстве случаев вы с самого начала просто пишете включение в список.

Шаблон, который вы должны увидеть, выглядит следующим образом:

Приведенный выше «образец» выражения-генератора соответствует следующей ниже функции-генератору:

In [39]:
def generator():
    for item in collection:
        yield expression

Точно так же, как и с включением в список, он дает вам типовой шаблон в стиле «формы для печенья», который можно применять ко многим функциям-генераторам с целью их преобразования в сжатые выражения-генераторы.

В этот шаблон можно добавить еще одно полезное дополнение, и это фильтрация элемента по условиям. Приведем пример: 

In [41]:
even_squares = (x * x for x in range(10) if x % 2 == 0)

Данный генератор порождает квадрат всех четных целых чисел от нуля до девяти. Фильтрующее условие с использованием оператора остатка %  отклонит любое значение, которое не делится на два: 

In [42]:
for x in even_squares:
    print(x)

0
4
16
36
64


Давайте обновим наш шаблон выражения-генератора. После добавления фильтрации элементов посредством условия if шаблон выглядит так: 

И снова этот шаблон соответствует относительно прямолинейной, но более длинной функции-генератору. Синтаксический сахар в своих лучших проявлениях:

In [43]:
def generator():
    for item in collection:
        if condition:
            yield expression

Поскольку выражения-генераторы являются, скажем так, выражениями, вы можете их использовать в одной строке вместе с другими инструкциями. Например, вы можете определить итератор и употребить его прямо на месте при помощи цикла for :

In [44]:
for x in ('Buongiorno' for i in range(3)):
    print(x)

Buongiorno
Buongiorno
Buongiorno


Есть и другой синтаксический трюк, который можно использовать для того, чтобы сделать выражения-генераторы красивее. Круглые скобки, окружающие выражение-генератор, могут быть опущены, если выражение-генератор используется в качестве единственного аргумента функции:

In [45]:
sum((x * 2 for x in range(10)))

90

In [46]:
# Сравните с:
sum(x * 2 for x in range(10))

90

Это позволяет писать сжатый и высокопроизводительный код. Поскольку выражения-генераторы генерируют значения «точно в срок» подобно тому, как это делает итератор на основе класса или функция-генератор, они эффективно используют оперативную память.

Выражения-генераторы являются полезным и питоновским инструментом в вашем наборе, но это не значит, что они должны использоваться для решения каждой задачи, с которой вы сталкиваетесь. В случае составных итераторов часто лучше написать функцию-генератор или даже итератор на основе класса.

Например, можно определить следующий ниже генератор, который производит серию целочисленных значений от одного до восьми, поддерживая нарастающий счетчик и выдавая новое значение всякий раз, когда с ним вызывается функция next() :

In [47]:
def integers():
    for i in range(1, 9):
        yield i

Вы можете подтвердить такое поведение, выполнив данный ниже фрагмент кода в интерпретаторе REPL Python:

In [48]:
chain = integers()
list(chain)

[1, 2, 3, 4, 5, 6, 7, 8]

Пока что не очень интересно. Но сейчас мы быстро это изменим. Дело в том, что генераторы могут быть «присоединены» друг к другу, благодаря чему можно строить эффективные алгоритмы обработки данных, которые работают как конвейер.
Вы можете взять «поток» значений, выходящих из генератора integers() , и направить их в еще один генератор. Например, такой, который принимает каждое число, возводит его в квадрат, а затем передает его дальше: 

In [49]:
def squared(seq):
    for i in seq:
        yield i * i

Это похоже на то, как работают конвейеры в UNIX. Мы состыковываем последовательность процессов в цепочку так, чтобы результат каждого процесса подавался непосредственно на вход следующего. Почему бы в наш конвейер не добавить еще один шаг, который инвертирует каждое значение, а потом передает его на следующий шаг обработки в цепи:

In [50]:
def negated(seq):
    for i in seq:
        yield -i

Если мы перестроим нашу цепочку генераторов и добавим negated в конец, то вот что мы получим на выходе: 

In [51]:
chain = negated(squared(integers()))
list(chain)

[-1, -4, -9, -16, -25, -36, -49, -64]

Моя любимая фишка формирования цепочки генераторов состоит в том, что обработка данных происходит по одному элементу за один раз. Буферизация между шагами обработки в цепочке отсутствует:

1. Генератор integers выдает одно-единственное значение, скажем, 3.
2. Это значение «активирует» генератор squared, который обрабатывает значение и передает его на следующую стадию как 3 × 3 = 9.
3. Квадрат целого числа, выданный генератором squared, немедленно передается в генератор negated, который модифицирует его в –9 и выдает его снова.

Вы можете продолжать расширять эту цепочку генераторов, чтобы отстроить конвейер обработки со многими шагами. Он по-прежнему будет выполняться эффективно и может легко быть модифицирован, потому что каждым шагом в цепочке является отдельная функция-генератор.

Каждая отдельная функция-генератор в этом конвейере обработки довольно сжатая. С помощью небольшой уловки мы можем сжать определение этого конвейера еще больше, не сильно жертвуя удобочитаемостью:

In [52]:
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)

Обратите внимание, как я заменил каждый шаг обработки в цепочке на выражение-генератор, строящийся на выходе из предыдущего шага.
Этот программный код эквивалентен цепочке генераторов, которые мы построили в этом разделе выше:

In [53]:
negated

<generator object <genexpr> at 0x7ff5bc207750>

In [54]:
list(negated)

[0, -1, -4, -9, -16, -25, -36, -49]

Единственным недостатком применения выражений-генераторов является то, что их не получится сконфигурировать с использованием аргументов функции и вы не сможете повторно использовать то же самое выражение-генератор многократно в том же самом конвейере обработки.

Но, безусловно, во время сборки конвейеров вы можете свободно комбинировать выражения-генераторы и обычные генераторы на свой вкус.

В случае с составными конвейерами это поможет улучшить удобочитаемость.